In [2]:
import pandas as pd
import zhconv
import os
import re

paths=[
    r'E:\年报文本\txts',
    r'E:\年报文本\predeals',
    r'E:\年报文本\ExtractResult',
    r'E:\年报文本\results'
]
files = [entry.name for entry in os.scandir(paths[0]) if entry.name.endswith('.txt')]
dataset = pd.DataFrame(files, columns=['文件名称'])
dataset['股票代码'] = dataset['文件名称'].apply(lambda x:x.split('-')[0])
dataset['截止日期'] = dataset['文件名称'].apply(lambda x:x.split('-')[1])
dataset['截止日期'] = dataset['截止日期'].astype('int')
dataset.sort_values(['股票代码','截止日期'],inplace=True)
# dataset

In [ ]:
# 首先提取2011之前的数据
dataset_2011 = dataset[(dataset['截止日期']<2012)&(dataset['截止日期']>2006)]
# 测试文件
exceptions=[]
for entryName in dataset_2011['文件名称']:
    path = os.path.join(paths[0], entryName)
    with open(path, 'r',encoding='utf8') as f:
        text =f.read().replace(' ','')
    text = zhconv.convert(text, 'zh-hans')
    res1 = re.finditer(r'\s+[\d第一二三四五六七八九十章节部分、．§]+\s*董+[事会局工作报告书\s]{4,}[^”―…⋯.\-—„\d\.\w]', text)
    res2 = re.finditer(r'\s+[第一二三四五六七八九十章节部分、．§\d]+\s*监+[事会局工作报告书\s]{4,}[^”―…⋯.\-—„\d\.\w]', text)
    res3 = re.finditer(r'[第一二三四五六七八九十章节部分]+[公司经营情况管理层]{3,7}讨论与分析\w*',re.sub('[\W\d]','',text[:2000]))
    target1 = [i for i in res1]
    target2 = [i for i in res2]
    target3 = [i for i in res3]
    if len(target3)>0:
        exceptions.append(entryName)
        print(f"-------{entryName}")
        continue
    elif len(target1) in [1,2,3] and len(target2) in [1,2,3]:
        text_ = text[target1[-1].start():target2[-1].start()]
    else:
        exceptions.append(entryName)
        print(len(target1), len(target2), entryName)
        continue
    with open(os.path.join(paths[2], entryName),'w',encoding='utf8') as f:
        f.write(text_)
with open('./datas/exceptions.txt','w',encoding='utf8') as f:
    f.write('\n'.join(exceptions))

In [ ]:
dataset_2020 = dataset[(dataset['截止日期']<2021)&(dataset['截止日期']>2011)]
exceptions=[]
for entryName in dataset_2020['文件名称']:
    path = os.path.join(paths[0], entryName)
    with open(path, 'r',encoding='utf8') as f:
        text =f.read().replace(' ','')
    text = zhconv.convert(text, 'zh-hans')
    res1 = re.finditer(r'\s+[第一二三四五六七八九十章节、．§\d]{2,5}\s*董+[事会局工作报告书\s]{4,}[^”―…⋯.\-—„\d\.\w]', text)
    res2 = re.finditer(r'\s+[第一二三四五六七八九十章节、．§\d]{2,5}\s*重[要大]事\s?项\s?[^”―…⋯.\-—„\d\.\w]', text)
    res3 = re.finditer(r'\s+[第一二三四五六七八九十章节、．§\d]{2,5}\s*[公司经营情况管理层]{3,7}讨论与分析\s?[^”―…⋯.\-—„\d\.\w]', text)
    target1 = [i for i in res1]
    target2 = [i for i in res2]
    target3 = [i for i in res3]
    if len(target3) in [1,2,3] and len(target2) in [1,2,3]:
        text_ = text[target3[-1].start():target2[-1].start()]
    elif len(target1) in [1,2,3] and len(target2) in [1,2,3]:
        text_ = text[target1[-1].start():target2[-1].start()]
    else:
        exceptions.append(entryName)
        print(entryName)
        continue
    with open(os.path.join(paths[2], entryName),'w',encoding='utf8') as f:
        f.write(text_)

with open('./datas/exceptions.txt','w',encoding='utf8') as f:
    f.write('\n'.join(exceptions))

In [4]:
dataset_2022 = dataset[dataset['截止日期']>2020]
exceptions=[]
for entryName in dataset_2022['文件名称']:
    path = os.path.join(paths[0], entryName)
    with open(path, 'r',encoding='utf8') as f:
        text =f.read().replace(' ','')
    text = zhconv.convert(text, 'zh-hans')
    res1 = re.finditer(r'\s+[第一二三四五六七八九十章节、/．]{2,5}\s*董+[事会局工作报告书\s]{4,}[^”―…⋯.\-—„\d\.\w]', text)
    res2 = re.finditer(r'\s+[第一二三四五六七八九十章节、/．]{2,5}\s*公司治理[报告及暨企业管治]*\s?[^”―…⋯.\-—„\d\.\w]', text)
    res3 = re.finditer(r'\s+[第一二三四五六七八九十章节、/．]{2,5}\s*[公司经营情况管理层]{3,7}讨论与分析\s?[^”―…⋯.\-—„\d\.\w]', text)
    target1 = [i for i in res1]
    target2 = [i for i in res2]
    target3 = [i for i in res3]
    if len(target3) in [1,2] and len(target2) in [1,2]:
        text_ = text[target3[-1].start():target2[-1].start()]
    elif len(target1) in [1,2] and len(target2) in [1,2]:
        text_ = text[target1[-1].start():target2[-1].start()]
    else:
        exceptions.append(entryName)
        print(len(target1), len(target2), len(target3), entryName)
        continue
    with open(os.path.join(paths[2], entryName),'w',encoding='utf8') as f:
        f.write(text_)

with open('./datas/exceptions.txt','w',encoding='utf8') as f:
    f.write('\n'.join(exceptions))

0 29 55 000001-2021-平安银行-年度报告.txt
0 28 54 000001-2022-平安银行-年度报告.txt
0 0 0 000032-2021-深桑达Ａ-年度报告.txt
0 0 0 000032-2022-深桑达Ａ-年度报告.txt
0 0 0 000034-2022-神州数码-年度报告.txt
0 0 0 000100-2021-TCL科技-年度报告.txt
0 34 27 000166-2022-申万宏源-年度报告.txt
0 0 0 000408-2022-藏格矿业-年度报告.txt
0 0 0 000426-2022-兴业矿业-年度报告.txt
0 0 0 000617-2021-中油资本-年度报告.txt
0 0 0 000617-2022-中油资本-年度报告.txt
0 0 0 000619-2021-海螺型材-年度报告.txt
0 0 0 000619-2022-海螺新材-年度报告.txt
0 0 0 000686-2021-东北证券-年度报告.txt
0 0 0 000686-2022-东北证券-年度报告.txt
0 0 0 000750-2021-国海证券-年度报告.txt
2 0 0 000756-2021-新华制药-年度报告.txt
2 0 0 000756-2022-新华制药-年度报告.txt
0 20 23 000878-2021-云南铜业-年度报告.txt
0 20 24 000878-2022-云南铜业-年度报告.txt
0 0 0 000900-2021-现代投资-年度报告.txt
0 1 0 000915-2022-华特达因-年度报告.txt
0 1 0 002028-2021-思源电气-年度报告.txt
0 22 50 002352-2021-顺丰控股-年度报告.txt
0 10 28 002352-2022-顺丰控股-年度报告.txt
0 0 0 002671-2022-龙泉股份-年度报告.txt
0 0 0 002736-2022-国信证券-年度报告.txt
0 0 0 002941-2021-新疆交建-年度报告.txt
0 0 0 002953-2021-日丰股份-年度报告.txt
0 15 22 003035-2022-南网能源-年度报告.txt
0 1 0 300054-2021-鼎龙股份-

In [ ]:
# 检测一下已经处理完毕的报告书中是否存在大小小于8kb的数据，删掉并拿出来与exception合并
files_result_ = [entry.name for entry in os.scandir(paths[2]) if entry.name.endswith('.txt')]
exceptions =[]
for file in files_result_:
    src = os.path.join(paths[2],file)
    size = os.path.getsize(src)
    if size<2000:
        print(size, file)
        exceptions.append(file)
        os.remove(src)
with open('./datas/exceptions.txt','w',encoding='utf8') as f:
    f.write('\n'.join(exceptions))

对于这些出现问题的文档，我们尽量提取，提取不出来也没事，就直接放弃吧。